# Lösung Übung 2 - Woche 3 und 4

Mit dem Wissen aus Woche 3 konnte man noch keine sehr zufriedenstellenden Ergebnisse erzielen. Das Problem hierbei war das schnelle Overfitting auf die Daten, da Dropout noch nicht bekannt war und auch Batch Normalization erst in Woche 4 vorgestellt wurde.

Da alle Bilder Hunde zeigen, die sich nicht so einfach voneinander unterscheiden lassen wie die Klassen im ImageNette Datensatz, war hier eine Genauigkeit von 40-50% zu erreichen.

Dieses Notebook stellt keine optimale Lösung dar, soll euch aber zeigen, welches grobe Netz gewählt werden kann.


In [0]:
%tensorflow_version 2.x

In [0]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Activation, Input, \
  Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from scipy.stats import reciprocal
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# jupyters magic command
%matplotlib inline

In [0]:
!pip install --upgrade deeplearning2020

  Using cached https://files.pythonhosted.org/packages/de/56/0b6adef8e6f5d89e9daa68e03d00850509f1553ce6303c0a49d7c619dd26/kerasltisubmission-0.4.9.tar.gz
  Created wheel for deeplearning2020: filename=deeplearning2020-0.4.21-py2.py3-none-any.whl size=8548 sha256=0ea0830d515fd509f6c003f900371260b49d96430d28f264643bbaae8ab7e93f
  Stored in directory: /root/.cache/pip/wheels/7f/c2/8a/f9f03fc839999f1fe9d5e5a9d2c97cdd5cb8329f61f82ea2c9
  Created wheel for kerasltisubmission: filename=kerasltisubmission-0.4.9-py2.py3-none-any.whl size=8867 sha256=46676efb4bbbe76f1b54684d83c45b469e905af7cf8d5966a5e4826eac478735
  Stored in directory: /root/.cache/pip/wheels/fd/61/f7/09171376b25408ae21b58e98c9fbf2eb924f676bb77659f983
Successfully built deeplearning2020 kerasltisubmission


In [0]:
from deeplearning2020.datasets import ImageWoof
train_data, test_data, classes= ImageWoof.load_data()


328294400/328288506 [==============================] - 8s 0us/step
/root/.keras/datasets/imagewoof2-320/train
Loaded 9025 images
/root/.keras/datasets/imagewoof2-320/val
Loaded 3929 images


## Loading and Preprocessing the data

In [0]:
# resize the images to a uniform size
def preprocess(image, label):
    resized_image = tf.image.resize(image, [300, 300])
    return resized_image, label

In [0]:


batch_size = 32
print('shape of training data before preprocessing: ', train_data)
train_data = train_data.shuffle(1000)


train_data = train_data.map(preprocess) \
  .batch(batch_size).prefetch(1)
test_data = test_data.map(preprocess) \
  .batch(batch_size).prefetch(1)
print('shape of training data after preprocessing: ', train_data)
print('shape of test data after preprocessing: ', test_data)

shape of training data before preprocessing:  <ParallelMapDataset shapes: ((None, None, 3), ()), types: (tf.float32, tf.int64)>
shape of training data after preprocessing:  <PrefetchDataset shapes: ((None, 300, 300, 3), (None,)), types: (tf.float32, tf.int64)>
shape of test data after preprocessing:  <PrefetchDataset shapes: ((None, 300, 300, 3), (None,)), types: (tf.float32, tf.int64)>


# Architektur des Netzes

In [0]:
# model
learning_rate=0.001
momentum=0.9
dense_neurons=500
n_filters=512
first_kernel_size=(7,7)

activation='elu'

# input size of images must be 300x300 with RGB color
input_layer = Input(shape=(300, 300, 3))

# Convolutional Neural Network
# It consists of 5 stacked Convolutional Layers with Max Pooling
model = Conv2D(
    filters=256,
    kernel_size=(7,7),
    activation=activation
)(input_layer)
model = MaxPooling2D((2,2))(model)

model = Conv2D(
    filters = 256, 
    kernel_size=(3,3), 
    activation=activation
)(model)
model = MaxPooling2D((2,2))(model)

model = Conv2D(
    filters = n_filters, 
    kernel_size=(3,3), 
    activation=activation
)(model)
model = MaxPooling2D((2,2))(model)

model = Conv2D(
    filters = n_filters, 
    kernel_size=(3,3), 
    activation=activation
)(model)
model = MaxPooling2D((2,2))(model)

model = Conv2D(filters = n_filters, 
  kernel_size=(3,3), 
  activation=activation, 
  padding='same'
)(model)
model = MaxPooling2D((2,2))(model)

model = Conv2D(filters = n_filters, 
  kernel_size=(3,3), 
  activation=activation, 
  padding='same'
)(model)
model = MaxPooling2D((2,2))(model)

model = Conv2D(filters = n_filters, 
  kernel_size=(3,3), 
  activation=activation, 
  padding='same'
)(model)

# Fully-Connected-Classifier
model = Flatten()(model)
model = Dense(
    dense_neurons,
    activation=activation
)(model)

model = Dense(
    dense_neurons / 2,
    activation='tanh'
)(model)

# Output Layer
output = Dense(10, activation="softmax")(model)

model = Model(input_layer, output)

# Compiling model
optimizer = keras.optimizers.SGD(lr=learning_rate, momentum=momentum)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 294, 294, 256)     37888     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 147, 147, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 145, 145, 256)     590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 72, 72, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 70, 70, 512)       1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 512)       0     

In [0]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    validation_data = test_data
)

Epoch 1/10
283/283 [==============================] - 123s 436ms/step - loss: 0.8194 - accuracy: 0.7309 - val_loss: 1.9239 - val_accuracy: 0.4100
Epoch 2/10
283/283 [==============================] - 123s 436ms/step - loss: 0.6151 - accuracy: 0.8079 - val_loss: 2.0038 - val_accuracy: 0.4123
Epoch 3/10
283/283 [==============================] - 123s 436ms/step - loss: 0.2568 - accuracy: 0.9378 - val_loss: 2.2265 - val_accuracy: 0.4065
Epoch 4/10
283/283 [==============================] - 123s 435ms/step - loss: 0.1921 - accuracy: 0.9591 - val_loss: 2.3152 - val_accuracy: 0.4093
Epoch 5/10
283/283 [==============================] - 123s 436ms/step - loss: 0.0354 - accuracy: 0.9981 - val_loss: 2.3080 - val_accuracy: 0.4291
Epoch 6/10
283/283 [==============================] - 123s 436ms/step - loss: 0.0127 - accuracy: 0.9998 - val_loss: 2.3591 - val_accuracy: 0.4383
Epoch 7/10
283/283 [==============================] - 123s 436ms/step - loss: 0.0079 - accuracy: 0.9999 - val_loss: 2.4098 -